In [16]:
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import Point
from sklearn.neighbors import KNeighborsClassifier


In [17]:
import pandas as pd

# Load the expanded dataset
df = pd.read_csv("expanded_soil_construction_dataset_1000.csv")

# Display the first few rows
df.head()


,Latitude,Longitude,Soil_Type,Green_Building_Suggestion,Reason,Seismic_Zone
0,13.125333,80.367732,Marshy,"Use pile foundation, flood-resistant materials...",Marshy land requires flood-resistant materials...,Zone II (Low Risk)
1,12.872860,77.629023,Loamy,"Adopt eco-friendly bricks, rooftop gardens, an...","Loamy soil is well-balanced, making it suitabl...",Zone II (Low Risk)
2,26.921343,75.854296,Loamy,"Adopt eco-friendly bricks, rooftop gardens, an...","Loamy soil is well-balanced, making it suitabl...",Zone V (Very High Risk)
3,12.873999,77.534966,Sandy,"Use reinforced foundation, solar panels, and w...","Sandy soil drains quickly, making it ideal for...",Zone II (Low Risk)
4,13.075770,80.316572,Sandy,"Use reinforced foundation, solar panels, and w...","Sandy soil drains quickly, making it ideal for...",Zone II (Low Risk)


In [18]:
from sklearn.preprocessing import LabelEncoder

# Encode seismic zones as numbers
le_zone = LabelEncoder()
df["Seismic_Zone_Num"] = le_zone.fit_transform(df["Seismic_Zone"])

# Features (Latitude, Longitude) and Target (Seismic Zone)
X = df[["Latitude", "Longitude"]]
y_zone = df["Seismic_Zone_Num"]


In [19]:
from sklearn.model_selection import train_test_split

# Split the dataset (80% training, 20% testing)
X_train, X_test, y_train_zone, y_test_zone = train_test_split(X, y_zone, test_size=0.2, random_state=42)


In [20]:
from sklearn.neighbors import KNeighborsClassifier

# Train KNN model for seismic zone classification
knn_zone = KNeighborsClassifier(n_neighbors=3)
knn_zone.fit(X_train, y_train_zone)


KNeighborsClassifier(n_neighbors=3)

In [21]:
from sklearn.metrics import accuracy_score

# Make predictions
y_pred_zone = knn_zone.predict(X_test)

# Calculate accuracy
accuracy_zone = accuracy_score(y_test_zone, y_pred_zone)
print(f"Seismic Zone Prediction Accuracy: {accuracy_zone * 100:.2f}%")


Seismic Zone Prediction Accuracy: 100.00%


In [22]:
import pickle

# Save the trained seismic zone model
with open("knn_zone_model.pkl", "wb") as zone_model_file:
    pickle.dump(knn_zone, zone_model_file)

# Save the Label Encoder for decoding seismic zones
with open("label_encoder_zone.pkl", "wb") as le_zone_file:
    pickle.dump(le_zone, le_zone_file)

print("Seismic Zone model and encoder saved successfully!")


Seismic Zone model and encoder saved successfully!


In [23]:
# Load the saved model and label encoder
with open("knn_zone_model.pkl", "rb") as model_file:
    knn_zone = pickle.load(model_file)

with open("label_encoder_zone.pkl", "rb") as encoder_file:
    le_zone = pickle.load(encoder_file)

# Function to predict seismic zone based on latitude & longitude
def predict_seismic_zone(lat, lon):
    zone_num = knn_zone.predict([[lat, lon]])[0]
    predicted_zone = le_zone.inverse_transform([zone_num])[0]
    return f"Predicted Seismic Zone: {predicted_zone}"

# Example usage
print(predict_seismic_zone(17.380, 78.382))


Predicted Seismic Zone: Zone II (Low Risk)


############################Round 3

In [15]:
# Load the saved model and label encoder
with open("knn_zone_model.pkl", "rb") as model_file:
    knn_zone = pickle.load(model_file)

with open("label_encoder_zone.pkl", "rb") as encoder_file:
    le_zone = pickle.load(encoder_file)

# Function to predict seismic zone based on latitude & longitude
def predict_seismic_zone(lat, lon):
    zone_num = knn_zone.predict([[lat, lon]])[0]
    predicted_zone = le_zone.inverse_transform([zone_num])[0]
    return f"Predicted Seismic Zone: {predicted_zone}"

# Example usage
print(predict_seismic_zone(28.704, 77.102))

NameError: name 'pickle' is not defined

In [24]:
import numpy as np
import pandas as pd
import pickle
with open("rf_green_building.pkl", "rb") as model_file:
    rf = pickle.load(model_file)
with open("label_encoder.pkl", "rb") as le_file:
    le_suggestion = pickle.load(le_file)
df = pd.read_csv("green_building_dataset_100000.csv")
reasons_dict = dict(zip(df["Green_Building_Suggestion"], df["Reason"]))
def predict_green_building(lat, lon, climate, elevation, hazard, temp, rainfall, urban_density, top_n=3):
    climate_map = {"Tropical": 0, "Dry": 1, "Temperate": 2, "Continental": 3}
    climate_num = climate_map.get(climate, 0)
    input_data = pd.DataFrame(np.array([[lat, lon, climate_num, elevation, hazard, temp, rainfall, urban_density]]),
                              columns=["Latitude", "Longitude", "Climate_Num", "Elevation", "Hazard_Risk",
                                       "Avg_Temperature", "Annual_Rainfall", "Urban_Density"])

    probabilities = rf.predict_proba(input_data)[0]  
    
    top_indices = np.argsort(probabilities)[-top_n:][::-1]
    top_suggestions = le_suggestion.inverse_transform(top_indices)

    
    recommendations = []
    for suggestion in top_suggestions:
        reason = reasons_dict.get(suggestion, "No specific reason available.")
        recommendations.append(f"{suggestion}: {reason}")

    return f"Top {top_n} Recommended Green Building Practices:\n" + "\n".join(recommendations)

# Example Prediction
print(predict_green_building(17.3850, 78.382, "Temprate", 500, 1, 32, 1200, 1))


Top 3 Recommended Green Building Practices:
Greywater Recycling System: Water scarcity regions benefit from recycling water for irrigation and flushing.
Solar Panels & Renewable Energy: High sunlight exposure makes solar power efficient for energy sustainability.
Energy-Efficient Windows & Insulation: Reducing heat gain in buildings lowers energy costs for cooling.


In [25]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier 

MODEL_PATH = "xgboost_model1.json" 
SCALER_PATH = "scaler1.pkl"  
ENCODER_PATH = "encoder1.pkl"  

file_path = "soil_construction_dataset_clean.csv"
df = pd.read_csv(file_path)

label_encoder = LabelEncoder()
df['Recommended_Foundation'] = label_encoder.fit_transform(df['Recommended_Foundation'])

joblib.dump(label_encoder, ENCODER_PATH)


x = df.drop(["Recommended_Foundation"], axis=1)
y = df['Recommended_Foundation']
categorical_cols = ["Soil_Type", "Seismic_Zone", "Flood_Risk"]


x = pd.get_dummies(x, columns=categorical_cols, drop_first=True)


model = XGBClassifier(random_state=42)
model.fit(x, y)


model.save_model(MODEL_PATH)


scaler1 = StandardScaler()
scaler1.fit(x)
joblib.dump(scaler1, SCALER_PATH)


def soil_improvement(foundation_type):
    foundation_type = foundation_type.lower()

    if foundation_type == "strip footing":
        print("\n🔹 **Soil Improvement for Strip Footing:**")
        print("✅ Compaction (Rollers/Rammers)")
        print("✅ Soil Replacement with Gravel/Sand")
        print("✅ Chemical Stabilization (Lime/Cement)")
        print("✅ Electro-Osmosis for Clay Soils")
        print("✅ Soil Nailing for Stability")

        print("\n🌍 **Soil Type:** Sandy or Loamy Soil with Medium Bearing Capacity")

        print("\n🌱 **Plants for Soil Improvement:** Vetiver Grass, Buffalo Grass")

        print("\n⚠ **Symptoms of Poor Soil Conditions:**")
        print("❌ Cracks in Soil Due to Settlement")
        print("❌ Waterlogging and Poor Drainage")
        print("❌ Loose, Unstable Soil that Erodes Easily")

    elif foundation_type == "raft foundation":
        print("\n🔹 **Soil Improvement for Raft Foundation:**")
        print("✅ Preloading with Vertical Drains")
        print("✅ Geotextiles & Geogrids for Reinforcement")
        print("✅ Stone Columns for Load Bearing")
        print("✅ Jet Grouting for Strengthening")
        print("✅ Deep Dynamic Compaction (Heavy Dropping Weights)")

        print("\n🌍 **Soil Type:** Soft Clay, Loose Sand, and Expansive Soil")

        print("\n🌱 **Plants for Soil Improvement:** Alfalfa, Clover, Saltgrass, Bermuda Grass")

        print("\n⚠ **Symptoms of Poor Soil Conditions:**")
        print("❌ Excessive Settlement of Structures")
        print("❌ Soil Expansion and Shrinkage (Causes Cracking)")
        print("❌ Water Seepage and Weak Bearing Capacity")

    elif foundation_type == "deep pile":
        print("\n🔹 **Soil Improvement for Deep Pile Foundation:**")
        print("✅ Grouting (Cement or Chemical) for Soil Strength")
        print("✅ Deep Soil Mixing (Lime/Cement Columns)")
        print("✅ Vacuum Consolidation for Clay Soils")
        print("✅ Micropiles & Geopiers for Load Distribution")
        print("✅ Electrokinetic Stabilization")

        print("\n🌍 **Soil Type:** Weak or Expansive Soils with Low Bearing Capacity")

        print("\n🌱 **Plants for Soil Improvement:** Willow Trees, Bamboo")

        print("\n⚠ **Symptoms of Poor Soil Conditions:**")
        print("❌ High Water Content Leading to Weak Soil")
        print("❌ Uneven Settling of Structures")
        print("❌ Deep Cracks Due to Weak Load-Bearing Capacity")

    elif foundation_type == "caisson foundation":
        print("\n🔹 **Soil Improvement for Caisson Foundation:**")
        print("✅ Stone Columns for Stability")
        print("✅ Underwater Grouting for Marine Structures")
        print("✅ Prefabricated Vertical Drains (PVDs)")
        print("✅ Artificial Island Creation for Stability")
        print("✅ Polymer Injection for Erosion Control")

        print("\n🌍 **Soil Type:** Waterlogged or Marine Clay Soil")

        print("\n🌱 **Plants for Soil Improvement:** Mangroves, Reed Grass, Marsh Plants")

        print("\n⚠ **Symptoms of Poor Soil Conditions:**")
        print("❌ Soil Erosion Due to Water Flow")
        print("❌ Structural Instability in Wet Conditions")
        print("❌ Poor Load-Bearing Capacity for Heavy Structures")

    else:
        print("\n❌ Invalid foundation type. Please choose from:")
        print("   - Strip Footing")
        print("   - Raft Foundation")
        print("   - Deep Pile")
        print("   - Caisson Foundation")


def process_soil_data(text):
    if text is None:
        return pd.DataFrame()

    data_dict = {}
    for line in text.strip().split("\n"):
        parts = line.split(":")
        if len(parts) == 2:
            key = parts[0].strip()
            value = parts[1].strip()
            try:
                data_dict[key] = float(value)
            except ValueError:
                data_dict[key] = value

    df = pd.DataFrame([data_dict])

    expected_columns = [
        'Latitude', 'Longitude', 'Moisture_Content', 'Bulk_Density', 'Plasticity_Index',
        'Bearing_Capacity', 'Cohesion', 'Angle_of_Friction', 'Groundwater_Level',
        'Soil_Type', 'Seismic_Zone', 'Flood_Risk'
    ]
    for col in expected_columns:
        if col not in df.columns:
            df[col] = 0  

    return df

loaded_model = XGBClassifier()
loaded_model.load_model(MODEL_PATH)
loaded_scaler = joblib.load(SCALER_PATH)
loaded_label_encoder = joblib.load(ENCODER_PATH)

soil_report_text = """
Latitude: 17.380
Longitude: 78.382
Moisture_Content: 35
Bulk_Density: 1.5
Plasticity_Index: 25
Bearing_Capacity: 90
Cohesion: 0.2
Angle_of_Friction: 22
Groundwater_Level: 3
Soil_Type: Peaty Soil
Seismic_Zone: IV
Flood_Risk: Yes
"""
soil_data = process_soil_data(soil_report_text)

if not soil_data.empty:
    
    soil_data_encoded = pd.get_dummies(soil_data, columns=categorical_cols, drop_first=True)

    
    expected_columns = loaded_scaler.feature_names_in_
    soil_data_encoded = soil_data_encoded.reindex(columns=expected_columns, fill_value=0)

    
    soil_data_encoded = soil_data_encoded.apply(pd.to_numeric, errors='coerce').fillna(0)

    
    soil_data_scaled = loaded_scaler.transform(soil_data_encoded)

    
    foundation_prediction = loaded_model.predict(soil_data_scaled)[0] 

    
    predicted_foundation = loaded_label_encoder.inverse_transform([foundation_prediction])[0]
    
    print(f"Recommended Foundation: {predicted_foundation}")
    
    foundation = predicted_foundation
    soil_improvement(foundation)

    

else:
    print("Error: Could not process soil data from text input.")


Recommended Foundation: Raft Foundation

🔹 **Soil Improvement for Raft Foundation:**
✅ Preloading with Vertical Drains
✅ Geotextiles & Geogrids for Reinforcement
✅ Stone Columns for Load Bearing
✅ Jet Grouting for Strengthening
✅ Deep Dynamic Compaction (Heavy Dropping Weights)

🌍 **Soil Type:** Soft Clay, Loose Sand, and Expansive Soil

🌱 **Plants for Soil Improvement:** Alfalfa, Clover, Saltgrass, Bermuda Grass

⚠ **Symptoms of Poor Soil Conditions:**
❌ Excessive Settlement of Structures
❌ Soil Expansion and Shrinkage (Causes Cracking)
❌ Water Seepage and Weak Bearing Capacity


In [26]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBRegressor

MODEL_PATH = "xgboost_model.json" 
SCALER_PATH = "scaler.pkl" 
ENCODER_PATH = "encoder.pkl" 


file_path = "soil_construction_dataset_clean.csv"
df = pd.read_csv(file_path)


numeric_df = df.select_dtypes(include=['number'])

quantiles = np.linspace(0, 1, 6) 
bins = np.interp(quantiles, np.linspace(0, 1, len(numeric_df["Suitability_Score"])), np.sort(numeric_df["Suitability_Score"]))
bins = np.unique(bins) 

num_bins = len(bins) - 1
labels = ["Very Low", "Low", "Medium", "High", "Very High"][:num_bins]
df["Suitability_Score"] = pd.cut(numeric_df["Suitability_Score"], bins=bins, labels=labels, include_lowest=True)
label_encoder = LabelEncoder()
df['Suitability_Score'] = label_encoder.fit_transform(df['Suitability_Score'])
x = df.drop(["Suitability_Score"], axis=1)
y = df['Suitability_Score']
categorical_cols = ["Soil_Type", "Seismic_Zone", "Flood_Risk", "Recommended_Foundation"]
x = pd.get_dummies(x, columns=categorical_cols, drop_first=True)
model = XGBRegressor(random_state=42)
model.fit(x, y)
model.save_model(MODEL_PATH)
scaler = StandardScaler()
scaler.fit(x)
joblib.dump(scaler, SCALER_PATH)
encoder_columns = x.columns  
joblib.dump(encoder_columns, ENCODER_PATH)
def process_soil_data(text):
    if text is None:
        return pd.DataFrame()

    data_dict = {}
    for line in text.strip().split("\n"):
        parts = line.split(":")
        if len(parts) == 2:
            key = parts[0].strip() 
            value = parts[1].strip() 
            try:
                data_dict[key] = float(value)
            except ValueError:
                data_dict[key] = value

    df = pd.DataFrame([data_dict])

    expected_columns = [
        'Latitude', 'Longitude', 'Moisture_Content', 'Bulk_Density', 'Plasticity_Index',
        'Bearing_Capacity', 'Cohesion', 'Angle_of_Friction', 'Groundwater_Level',
        'Soil_Type', 'Seismic_Zone', 'Flood_Risk', "Recommended_Foundation"
    ]
    for col in expected_columns:
        if col not in df.columns:
            df[col] = 0 

    return df

loaded_model = XGBRegressor()
loaded_model.load_model(MODEL_PATH)
loaded_scaler = joblib.load(SCALER_PATH)
loaded_encoder_columns = joblib.load(ENCODER_PATH)  

soil_report_text = """
Latitude: 17.382
Longitude: 78.382
Moisture_Content: 35
Bulk_Density: 1.8
Plasticity_Index: 25
Bearing_Capacity: 90
Cohesion: 0.15
Angle_of_Friction: 22
Groundwater_Level: 1.5
Soil_Type: Sandy Clay
Seismic_Zone: II
Flood_Risk: No
Recommended_Foundation: Deep Pile
"""


soil_data = process_soil_data(soil_report_text)

if not soil_data.empty:
    soil_data_encoded = pd.get_dummies(soil_data, columns=categorical_cols, drop_first=True)
    expected_columns = loaded_scaler.feature_names_in_  # Ensures exact feature match
    soil_data_encoded = soil_data_encoded.reindex(columns=expected_columns, fill_value=0)
    soil_data_encoded = soil_data_encoded.apply(pd.to_numeric, errors='coerce').fillna(0)
    soil_data_scaled = loaded_scaler.transform(soil_data_encoded)
    suitability_score = loaded_model.predict(soil_data_scaled)[0]  # Access first prediction
    predicted_category = label_encoder.inverse_transform([int(round(suitability_score))])
    
    print(f"Predicted Suitability Score: {suitability_score} ({predicted_category[0]})")

else:
    print("Error: Could not process soil data from text input.")


Predicted Suitability Score: 1.8116953372955322 (Medium)


In [27]:

loaded_model = joblib.load("xgb_zone_prediction_model3.pkl")


loaded_scaler = joblib.load("scaler3.pkl")


loaded_label_encoders = joblib.load("label_encoders3.pkl")


loaded_y_encoder = joblib.load("y_encoder3.pkl")

def predict_zone(latitude, longitude, land_use, soil_type, hazard_risk, population_density):
    input_data = pd.DataFrame({
        "Latitude": [latitude],
        "Longitude": [longitude],
        "Land Use": [loaded_label_encoders["Land Use"].transform([land_use])[0] if land_use in loaded_label_encoders["Land Use"].classes_ else 0],
        "Soil Type": [loaded_label_encoders["Soil Type"].transform([soil_type])[0] if soil_type in loaded_label_encoders["Soil Type"].classes_ else 0],
        "Hazard Risk": [loaded_label_encoders["Hazard Risk"].transform([hazard_risk])[0] if hazard_risk in loaded_label_encoders["Hazard Risk"].classes_ else 0],
        "Population Density": [population_density]
    })
    
    
    input_data[["Latitude", "Longitude", "Population Density"]] = loaded_scaler.transform(input_data[["Latitude", "Longitude", "Population Density"]])
    
    
    prediction = loaded_model.predict(input_data)
    
    return loaded_y_encoder.inverse_transform(prediction)[0]

predicted_zone = predict_zone(17.380, 78.382, "Residential", "Red Sandy", "Low", 15000)
print("Predicted Zone:", predicted_zone)



Predicted Zone: Heavy Industrial
